In [ ]:
### 使用常见分类器对数据集初步评估

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import IsolationForest

In [4]:
# data = pd.read_csv("E:\竞赛\服创赛-A08\data\process_data.csv")
data = pd.read_csv("../../data/new_features.csv")

if 'Unnamed: 0' in data.columns:
    data = data.drop(['Unnamed: 0','个人编码'], axis=1)
else:
    data = data.drop('个人编码', axis=1)

In [5]:
# 筛选RES列值为1的数据
data_res_1 = data[data['RES'] == 1]

# 计算RES为1的数据中每列的缺失值数量
missing_values_count_res_1 = data_res_1.isnull().sum()

# 过滤出有缺失值的列
missing_values_res_1 = missing_values_count_res_1[missing_values_count_res_1 > 0]

# 打印有缺失值的列和对应的缺失值数量（当RES列的值为1）
print("当RES列的值为1时，有缺失值的列及其缺失数量：")
print(missing_values_res_1)

# 构建的特征集中含有缺失值，部分分类器对缺失值敏感，删除样本
data = data.dropna()

当RES列的值为1时，有缺失值的列及其缺失数量：
月就诊天数_AVG_病种费用标准差值            3
月就诊医院数_MAX_病种费用标准差值           3
月就诊医院数_AVG_病种费用标准差值           3
就诊次数_SUM_病种费用标准差值             3
月就诊次数_MAX_病种费用标准差值            3
月就诊次数_AVG_病种费用标准差值            3
月统筹金额_MAX_病种费用标准差值            3
月统筹金额_AVG_病种费用标准差值            3
月药品金额_MAX_病种费用标准差值            3
月药品金额_AVG_病种费用标准差值            3
医院_就诊天数_MAX_病种费用标准差值          3
医院_就诊天数_AVG_病种费用标准差值          3
医院_统筹金_MAX_病种费用标准差值           3
医院_统筹金_AVG_病种费用标准差值           3
医院_药品_MAX_病种费用标准差值            3
医院_药品_AVG_病种费用标准差值            3
个人账户金额_SUM_病种费用标准差值           3
统筹支付金额_SUM_病种费用标准差值           3
ALL_SUM_病种费用标准差值              3
可用账户报销金额_SUM_病种费用标准差值         3
药品费发生金额_SUM_病种费用标准差值          3
药品费自费金额_SUM_病种费用标准差值          3
药品费申报金额_SUM_病种费用标准差值          3
贵重药品发生金额_SUM_病种费用标准差值         3
中成药费发生金额_SUM_病种费用标准差值         3
中草药费发生金额_SUM_病种费用标准差值         3
检查费发生金额_SUM_病种费用标准差值          3
检查费自费金额_SUM_病种费用标准差值          3
检查费申报金额_SUM_病种费用标准差值          3
贵重检查费金额_SUM_病种费用标准差值          3
治疗费发生金额_SUM_病种费

In [6]:
X = data.drop('RES', axis=1)
y = data['RES']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# RF

In [25]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_probabilities = rf_model.predict_proba(X_test)[:, 1]
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_predictions))
print("Random Forest AUC Score:", roc_auc_score(y_test, rf_probabilities))

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3791
           1       0.84      0.35      0.50       209

    accuracy                           0.96      4000
   macro avg       0.90      0.68      0.74      4000
weighted avg       0.96      0.96      0.96      4000

Random Forest AUC Score: 0.9091199378028294


In [26]:
rf_predictions = rf_model.predict(X_train)
rf_probabilities = rf_model.predict_proba(X_train)[:, 1]
print("Random Forest Classification Report:")
print(classification_report(y_train, rf_predictions))
print("Random Forest AUC Score:", roc_auc_score(y_train, rf_probabilities))

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11416
           1       1.00      1.00      1.00       581

    accuracy                           1.00     11997
   macro avg       1.00      1.00      1.00     11997
weighted avg       1.00      1.00      1.00     11997

Random Forest AUC Score: 1.0


# GBDT

In [27]:
gbdt_model = GradientBoostingClassifier(random_state=42)
gbdt_model.fit(X_train, y_train)
gbdt_predictions = gbdt_model.predict(X_test)
gbdt_probabilities = gbdt_model.predict_proba(X_test)[:, 1]
print("GBDT Classification Report:")
print(classification_report(y_test, gbdt_predictions))
print("GBDT AUC Score:", roc_auc_score(y_test, gbdt_probabilities))

GBDT Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      3791
           1       0.76      0.35      0.48       209

    accuracy                           0.96      4000
   macro avg       0.86      0.67      0.73      4000
weighted avg       0.95      0.96      0.95      4000

GBDT AUC Score: 0.9153693146321116


In [28]:
gbdt_predictions = gbdt_model.predict(X_train)
gbdt_probabilities = gbdt_model.predict_proba(X_train)[:, 1]
print("GBDT Classification Report:")
print(classification_report(y_train, gbdt_predictions))
print("GBDT AUC Score:", roc_auc_score(y_train, gbdt_probabilities))

GBDT Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99     11416
           1       0.93      0.49      0.65       581

    accuracy                           0.97     11997
   macro avg       0.95      0.75      0.82     11997
weighted avg       0.97      0.97      0.97     11997

GBDT AUC Score: 0.9643854474862108


# XGBoost

In [29]:
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
xgb_probabilities = xgb_model.predict_proba(X_test)[:, 1]
print("XGBoost Classification Report:")
print(classification_report(y_test, xgb_predictions))
print("XGBoost AUC Score:", roc_auc_score(y_test, xgb_probabilities))

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      3791
           1       0.70      0.33      0.45       209

    accuracy                           0.96      4000
   macro avg       0.83      0.66      0.71      4000
weighted avg       0.95      0.96      0.95      4000

XGBoost AUC Score: 0.9256776626586009


In [30]:
xgb_predictions = xgb_model.predict(X_train)
xgb_probabilities = xgb_model.predict_proba(X_train)[:, 1]
print("XGBoost Classification Report:")
print(classification_report(y_train, xgb_predictions))
print("XGBoost AUC Score:", roc_auc_score(y_train, xgb_probabilities))

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11416
           1       1.00      1.00      1.00       581

    accuracy                           1.00     11997
   macro avg       1.00      1.00      1.00     11997
weighted avg       1.00      1.00      1.00     11997

XGBoost AUC Score: 1.0


# lightGBM

In [31]:
lgb_model = LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)
lgb_probabilities = lgb_model.predict_proba(X_test)[:, 1]
print("LightGBM Classification Report:")
print(classification_report(y_test, lgb_predictions))
print("LightGBM AUC Score:", roc_auc_score(y_test, lgb_probabilities))

[LightGBM] [Info] Number of positive: 581, number of negative: 11416
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.066938 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 90724
[LightGBM] [Info] Number of data points in the train set: 11997, number of used features: 461
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.048429 -> initscore=-2.978020
[LightGBM] [Info] Start training from score -2.978020
LightGBM Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      3791
           1       0.71      0.33      0.45       209

    accuracy                           0.96      4000
   macro avg       0.84      0.66      0.71      4000
weighted avg       0.95      0.96      0.95      4000

LightGBM AUC Score: 0.9277689920347738


In [32]:
lgb_predictions = lgb_model.predict(X_train)
lgb_probabilities = lgb_model.predict_proba(X_train)[:, 1]
print("LightGBM Classification Report:")
print(classification_report(y_train, lgb_predictions))
print("LightGBM AUC Score:", roc_auc_score(y_train, lgb_probabilities))

LightGBM Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11416
           1       1.00      1.00      1.00       581

    accuracy                           1.00     11997
   macro avg       1.00      1.00      1.00     11997
weighted avg       1.00      1.00      1.00     11997

LightGBM AUC Score: 1.0


# 决策树

In [10]:
dt_model = DecisionTreeClassifier(random_state=42,max_depth=7,min_samples_split=10,min_samples_leaf=10)
dt_model.fit(X_train, y_train)
dt_predictions = dt_model.predict(X_test)
dt_probabilities_test = dt_model.predict_proba(X_test)[:, 1]

print("Decision Tree Classification Report:")
print(classification_report(y_test, dt_predictions))
print("Decision Tree AUC Score:", roc_auc_score(y_test, dt_probabilities_test))

Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      3791
           1       0.62      0.34      0.44       209

    accuracy                           0.95      4000
   macro avg       0.79      0.67      0.71      4000
weighted avg       0.95      0.95      0.95      4000

Decision Tree AUC Score: 0.8200421799805381


In [11]:
dt_predictions_train = dt_model.predict(X_train)
dt_probabilities_train = dt_model.predict_proba(X_train)[:, 1]

print("Decision Tree Classification Report:")
print(classification_report(y_train, dt_predictions_train))
print("Decision Tree AUC Score:", roc_auc_score(y_train, dt_probabilities_train))

Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     11416
           1       0.80      0.47      0.59       581

    accuracy                           0.97     11997
   macro avg       0.89      0.73      0.79     11997
weighted avg       0.97      0.97      0.96     11997

Decision Tree AUC Score: 0.8947736335270002


# 多层感知机

In [47]:
mlp_model = make_pipeline(StandardScaler(), 
                          MLPClassifier(hidden_layer_sizes=(128,128,128,64,32,16), 
                                        max_iter=300, 
                                        activation='relu', 
                                        solver='adam', 
                                        random_state=42))
mlp_model.fit(X_train, y_train)

# Making predictions and calculating probabilities
mlp_predictions = mlp_model.predict(X_test)
mlp_probabilities = mlp_model.predict_proba(X_test)[:, 1]

# Evaluating the model
print("MLP Classification Report:")
print(classification_report(y_test, mlp_predictions))
print("MLP AUC Score:", roc_auc_score(y_test, mlp_probabilities))

MLP Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      3791
           1       0.60      0.34      0.44       209

    accuracy                           0.95      4000
   macro avg       0.78      0.67      0.71      4000
weighted avg       0.95      0.95      0.95      4000

MLP AUC Score: 0.8619558536397587


In [48]:
mlp_predictions = mlp_model.predict(X_train)
mlp_probabilities = mlp_model.predict_proba(X_train)[:, 1]

# Evaluating the model
print("MLP Classification Report:")
print(classification_report(y_train, mlp_predictions))
print("MLP AUC Score:", roc_auc_score(y_train, mlp_probabilities))

MLP Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11416
           1       0.98      0.90      0.94       581

    accuracy                           0.99     11997
   macro avg       0.99      0.95      0.97     11997
weighted avg       0.99      0.99      0.99     11997

MLP AUC Score: 0.999036138547583


In [66]:
data = pd.read_csv("E:\竞赛\服创赛-A08\\data\特征集\\new_features_data_2.csv")
data = data.dropna()

if 'Unnamed: 0' in data.columns:
    data = data.drop(['Unnamed: 0','个人编码'], axis=1)
else:
    data = data.drop('个人编码', axis=1)

X = data.drop('RES', axis=1)
y = data['RES']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Logistic

In [53]:
log_model = make_pipeline(StandardScaler(), LogisticRegression(random_state=42))
log_model.fit(X_train, y_train)
log_predictions = log_model.predict(X_test)
log_probabilities = log_model.predict_proba(X_test)[:, 1]

print("Logistic Regression Classification Report:")
print(classification_report(y_test, log_predictions))
print("Logistic Regression AUC Score:", roc_auc_score(y_test, log_probabilities))

D:\Anaconda3\envs\A08\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      3791
           1       0.66      0.38      0.48       209

    accuracy                           0.96      4000
   macro avg       0.81      0.69      0.73      4000
weighted avg       0.95      0.96      0.95      4000

Logistic Regression AUC Score: 0.8509381953480857


In [54]:
log_predictions = log_model.predict(X_train)
log_probabilities = log_model.predict_proba(X_train)[:, 1]

print("Logistic Regression Classification Report:")
print(classification_report(y_train, log_predictions))
print("Logistic Regression AUC Score:", roc_auc_score(y_train, log_probabilities))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99     11416
           1       0.91      0.50      0.64       581

    accuracy                           0.97     11997
   macro avg       0.94      0.75      0.81     11997
weighted avg       0.97      0.97      0.97     11997

Logistic Regression AUC Score: 0.9479751521854765


# 隔离森林

In [67]:
print(data['RES'].value_counts())
data['RES'] = data['RES'].map({1: -1, 0: 1})
data['RES'].value_counts()

RES
0    15207
1      790
Name: count, dtype: int64


RES
 1    15207
-1      790
Name: count, dtype: int64

In [68]:
X = data.drop('RES', axis=1)
y = data['RES']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [69]:
iso_forest = IsolationForest(n_estimators=100, random_state=42)
iso_forest.fit(X_train)


y_pred_train = iso_forest.predict(X_train)
y_pred_test = iso_forest.predict(X_test)

# Evaluating the model
print("Isolation Forest Classification Report on Test Data:")
print(classification_report(y_test, y_pred_test))

Isolation Forest Classification Report on Test Data:
              precision    recall  f1-score   support

          -1       0.42      0.35      0.38       209
           1       0.96      0.97      0.97      3791

    accuracy                           0.94      4000
   macro avg       0.69      0.66      0.68      4000
weighted avg       0.94      0.94      0.94      4000


In [70]:
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

          -1       0.42      0.37      0.39       581
           1       0.97      0.97      0.97     11416

    accuracy                           0.94     11997
   macro avg       0.69      0.67      0.68     11997
weighted avg       0.94      0.94      0.94     11997
